In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

## Подключение к БД

В файле `config/.env` необходимо задать все параметры для подключения к бд

In [ ]:
# путь к конфигурационному файлу
load_dotenv('../config/.env')
    
config = {
        'host': os.getenv('DB_HOST'),
        'user': os.getenv('DB_USER'),
        'password': os.getenv('DB_PASSWORD'),
        'database': os.getenv('DB_NAME'),
        'port': os.getenv('DB_PORT', 3306)
    }


In [ ]:
connection_string = (f"mysql+mysqlconnector://{config['user']}:{config['password']}@{config['host']}:{config['port']}/{config['database']}")

Поскольку запросы в ноутбуке выполняются с перерывами, подключение может быть разорвано. Поэтому каждый запрос обернем в функцию, которая возобновляет подключение:

In [ ]:
def get_query(query):
    """Создает новое подключение для каждого запроса"""
    engine = create_engine(connection_string)
    try:
        with engine.connect() as conn: 
            result = pd.read_sql_query(query, conn)
            return result
    finally:
        engine.dispose()  # Важно: закрываем engine

## Аналитика

### 1. Количество пользователей, которые ни разу не заходили в систему

In [ ]:
query_1 = """SELECT COUNT(*) as Amount
             FROM users
            WHERE last_visited_step_id IS NULL"""

result_1  = get_query(query_1)


print(f'Количество пользователей, которые никогда не заходили в систему: {result_1.iat[0,0]}')

###  2. Количество пользователей, которые заходили в систему, но ничего не пытались решить

In [ ]:
query_2 = """SELECT COUNT(id) as Amount
FROM users
WHERE last_visited_step_id IS NOT NULL AND id NOT IN 
(SELECT DISTINCT user_id  FROM solutions)"""

result_2  = get_query(query_2)
print(f'Количество пользователей, которые заходили в систему, но ничего не пытались решить: {result_2.iat[0,0]}')

### 3. Рейтинг студентов
Теперь не бездельники: таблица с количеством правильно решенных заданий и количество попыток решений (неважно, правильных в итоге или нет) для каждого пользователя

In [ ]:
query_3 = """SELECT first.login, first.attempts_total, second.completed_steps
FROM
(SELECT u.login, SUM(s.attempts_count) AS attempts_total
FROM solutions s
LEFT JOIN users u ON s.user_id = u.id
GROUP BY s.user_id) AS first LEFT JOIN    -- суммарное количество попыток решений
(SELECT u.login, COUNT(step_id) AS completed_steps
 FROM solutions s
 LEFT JOIN users u ON s.user_id = u.id
 WHERE is_correct = 1
 GROUP BY s.user_id) AS second            -- количество решенных заданий
 ON first.login = second.login
 ORDER BY completed_steps DESC"""
result_3 = get_query(query_3)
result_3

### 4. Аналитика по степам
Количество попыток, правильных решений (= количество решивших пользователей) и процент правильных решений для каждого степа. Отфильтрованы только с малой долей правильных решений (<30%). К степам также добавлены названия уроков

In [ ]:
query_4 = """SELECT lessons.title as lesson, steps.id as step, SUM(solutions.attempts_count) as attempts_total, SUM(solutions.is_correct) as correct_total, ROUND(SUM(solutions.is_correct) / SUM(solutions.attempts_count) * 100, 1) as proportion_of_correct
FROM steps
LEFT JOIN solutions ON steps.id = solutions.step_id
LEFT JOIN lessons on steps.lesson_id = lessons.id
WHERE steps.category_id != 1
GROUP BY steps.id
HAVING proportion_of_correct < 30
ORDER BY proportion_of_correct"""

result_4 = get_query(query_4)

result_4

### 5. В каком уроке меньше всего доля верных решений из всех попыток?

In [ ]:
query_5 = """SELECT sections.title as section, lessons.title as lesson, SUM(solutions.attempts_count) as attempts_total, SUM(solutions.is_correct) as correct_total, ROUND(SUM(solutions.is_correct) / SUM(solutions.attempts_count) * 100, 0) as proportion_of_correct
FROM steps
LEFT JOIN solutions ON steps.id = solutions.step_id
LEFT JOIN lessons on steps.lesson_id = lessons.id
LEFT JOIN sections ON lessons.section_id = sections.id
WHERE steps.category_id != 1
GROUP BY lesson
ORDER BY proportion_of_correct -- пропорция верно решенных заданий"""


result_5 = get_query(query_5)
result_5

### 6. Шаги, которые никто никогда не проходил и не просматривал
(за исключением админа, который в любом случае проверяет все степы)

In [ ]:
query_6 = """SELECT lessons.title as lesson, steps.id as step_id
FROM steps
LEFT JOIN lessons ON steps.lesson_id = lessons.id
WHERE steps.id NOT IN (SELECT DISTINCT(step_views.step_id)
				 FROM step_views
				 LEFT JOIN users ON step_views.user_id = users.id 
				 WHERE users.login != 'admin')                      
AND steps.id NOT IN (SELECT DISTINCT solutions.step_id
			   FROM solutions
			   LEFT JOIN users ON solutions.user_id = users.id
			   WHERE users.login != 'admin')"""

result_6 = get_query(query_6)

result_6